In [104]:
import requests
import pandas as pd

token = "243c4775727d1020f3088e838b2cd789"

id_dh = 1014700
id_dede = 28244
id_mafe = 83270

url = "https://api.start.gg/gql/alpha"

# Headers com o token de API incluído
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json',
    'Cookie': 'MUID=245FA46249BC61152B99B08E48AE60F8; _EDGE_S=F=1; _EDGE_V=1'
}

In [86]:
# Consulta GraphQL com variáveis pra pegar os entrantes de um evento
query_entrantes = """
query EventStandings($eventId: ID!, $page: Int!, $perPage: Int!) {
  event(id: $eventId) {
    id
    name
    standings(query: {
      perPage: $perPage,
      page: $page
    }){
      nodes {
        placement
        entrant {
          id
          name
        }
      }
    }
  }
}
"""
#Consulta GraphQL com variáveis pra pegar os owner de um evento

query_id_owner = """

query TournamentOwner($slug: String!) {
  tournament(slug: $slug) {
    id
    name
    owner {
      id
      name
    }
  }
}

"""


#Consulta GraphQL com variáveis pra pegar os eventos de um owner

query_eventos_owner = """

query TournamentsByOwner($perPage: Int!, $ownerId: ID!) {
    tournaments(query: {
      perPage: $perPage
      filter: {
        ownerId: $ownerId
      }
    }) {
    nodes {
      id
      name
      slug
    }
  }
},
"""


In [144]:
# Variáveis que serão passadas na consulta
variables = {
  #"eventId": 618643,
  "page": 1,
  "perPage": 10,
  "ownerId" : id_dede
  #"slug": "opera-o-o-verdadeiro-2" 
}

# Construir o payload da requisição
payload = {
    "query": query_eventos_owner,
    "variables": variables
}

# Fazer a requisição POST com o corpo corretamente formatado
response = requests.post(url, json=payload, headers=headers)

# Exibir a resposta da API
print(response.text)

{"data":{"tournaments":{"nodes":[{"id":699958,"name":"Taverna do Smash Setembro 2024","slug":"tournament\/taverna-do-smash-setembro-2024"},{"id":688632,"name":"Taverna do Smash Agosto 2024","slug":"tournament\/taverna-do-smash-agosto-2024"},{"id":618644,"name":"Dela\u00e7\u00e3o Premiada 2024","slug":"tournament\/dela-o-premiada-2024"},{"id":618643,"name":"Circuito Paulista de i-Smash Junho\/2024","slug":"tournament\/circuito-paulista-de-i-smash-junho-2024"},{"id":618642,"name":"Circuito Paulista de i-Smash Maio\/2024","slug":"tournament\/circuito-paulista-de-i-smash-maio-2024"},{"id":618641,"name":"Circuito Paulista de i-Smash Abril\/2024","slug":"tournament\/circuito-paulista-de-i-smash-abril-2024"},{"id":647452,"name":"Opera\u00e7\u00e3o: O Verdadeiro 2","slug":"tournament\/opera-o-o-verdadeiro-2"},{"id":618640,"name":"Circuito Paulista de i-Smash Mar\u00e7o\/2024","slug":"tournament\/circuito-paulista-de-i-smash-mar-o-2024"},{"id":618638,"name":"Opera\u00e7\u00e3o: O Verdadeiro","s

In [ ]:
# Variáveis que serão passadas na consulta
variables = {
  "eventId": 618643,
  "page": 1,
  "perPage": 10,
  "ownerId" : id_dede
  #"slug": "opera-o-o-verdadeiro-2" 
}

# Construir o payload da requisição
payload = {
    "query": query_eventos_owner,
    "variables": variables
}

# Fazer a requisição POST com o corpo corretamente formatado
response = requests.post(url, json=payload, headers=headers)

# Exibir a resposta da API
print(response.text)

In [145]:
data = response.json()
print(data)

{'data': {'tournaments': {'nodes': [{'id': 699958, 'name': 'Taverna do Smash Setembro 2024', 'slug': 'tournament/taverna-do-smash-setembro-2024'}, {'id': 688632, 'name': 'Taverna do Smash Agosto 2024', 'slug': 'tournament/taverna-do-smash-agosto-2024'}, {'id': 618644, 'name': 'Delação Premiada 2024', 'slug': 'tournament/dela-o-premiada-2024'}, {'id': 618643, 'name': 'Circuito Paulista de i-Smash Junho/2024', 'slug': 'tournament/circuito-paulista-de-i-smash-junho-2024'}, {'id': 618642, 'name': 'Circuito Paulista de i-Smash Maio/2024', 'slug': 'tournament/circuito-paulista-de-i-smash-maio-2024'}, {'id': 618641, 'name': 'Circuito Paulista de i-Smash Abril/2024', 'slug': 'tournament/circuito-paulista-de-i-smash-abril-2024'}, {'id': 647452, 'name': 'Operação: O Verdadeiro 2', 'slug': 'tournament/opera-o-o-verdadeiro-2'}, {'id': 618640, 'name': 'Circuito Paulista de i-Smash Março/2024', 'slug': 'tournament/circuito-paulista-de-i-smash-mar-o-2024'}, {'id': 618638, 'name': 'Operação: O Verdade

In [136]:
standings = data['data']['event']['standings']['nodes']
nome_evento = data['data']["event"]["name"]
id_evento = data['data']["event"]["id"]

#torneios = data["data"]["tournaments"]["nodes"]

df = pd.json_normalize(standings)

#df["nome evento"] = nome_evento 
#df["id evento"] = id_evento

df

""


In [118]:
ids_torneio = df["id"].tolist()
ids_torneio

[699958,
 688632,
 618644,
 618643,
 618642,
 618641,
 647452,
 618640,
 618638,
 600156]

In [148]:
dados = pd.DataFrame()

for i in ids_torneio:
    variables = {
      "eventId": i,
      "page": 1,
      "perPage": 10,
    } 

    payload = {
        "query": query_entrantes,
        "variables": variables
    }
    
    # Fazer a requisição POST com o corpo corretamente formatado
    response = requests.post(url, json=payload, headers=headers)

    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        standings = data['data']['event']['standings']['nodes']
        
        # Adiciona uma coluna para o ID do torneio e nome
        for standing in standings:
            standing['event_id'] = data['data']['event']['id']
            standing['event_name'] = data['data']['event']['name']
        
    else:
        print(f"Erro: {response.status_code}, {response.text}")

    data = response.json()
    standings = data['data']['event']['standings']['nodes']
    nome_evento = data['data']["event"]["name"]
    id_evento = data['data']["event"]["id"]
    
    df = pd.json_normalize(standings)
    
    df["nome evento"] = nome_evento 
    df["id evento"] = id_evento

    dados = pd.concat([dados,df])


    
display(dados)

NameError: name 'all_standings' is not defined